In [3]:
import numpy as np 
import pandas as pd 
import joblib
import requests
import json
import time

In [10]:
# Set pandas to show max columns and max rows
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [4]:
from sklearn.preprocessing import MultiLabelBinarizer

In [5]:
# This key expires every 24 hours (email: akshay.n.bhasin@gmail.com for another key)
Developer_API_Key='RGAPI-067415c3-419f-4dcf-a2d1-8ff9ce7b414e' 

# Load the optimized model
model = joblib.load('Optimized Random Forest Classifier Model.pkl')

# Enter your name
Summoner_name = 'Mmmscruffy'

# Request the encrypted account id
r = requests.get('https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + Summoner_name + '?api_key=' + Developer_API_Key)
try:
    account_id = pd.json_normalize(r.json())['accountId'][0]
except:
    print(f"Your ID doesn't exist :(")

c:\users\user\anaconda3\envs\streamlit\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\user\anaconda3\envs\streamlit\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\user\anaconda3\envs\streamlit\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [6]:
#  Request a list of all gameIDs with Nasus
r = requests.get('https://na1.api.riotgames.com/lol/match/v4/matchlists/by-account/' + account_id + '?champion=75&api_key=' + Developer_API_Key)
gameIDs_df = pd.json_normalize(r.json(), 'matches')
# GameId is a float, this needs to by an integer, convert...
gameIDs_df["gameId"] = pd.to_numeric(gameIDs_df["gameId"], downcast='integer')
game_IDs = gameIDs_df["gameId"].tolist()
game_IDs

[3568014258,
 3561518103,
 3558332940,
 3557281991,
 3557237479,
 3557195366,
 3557120817,
 3556997835,
 3554747992,
 3554736615,
 3554713511,
 3554669201,
 3554626568,
 3548461358,
 3548430496,
 3548258833,
 3547290879,
 3547241811,
 3538079664,
 3537574969,
 3529761284,
 3526950747,
 3525228929,
 3525175496,
 3524942576,
 3524226528,
 3523583458,
 3523510436,
 3522828846,
 3522873633,
 3522758925,
 3515101691,
 3514009453,
 3510081299,
 3490769845,
 3060091218]

In [7]:
# Send another get request to get the match data
r = requests.get('https://na1.api.riotgames.com/lol/match/v4/matches/' + '3568014258' + '?api_key=' + Developer_API_Key)

# Create the dataframes (Participant and Bans)
part_matchdat_df=pd.json_normalize(data=r.json(), record_path='participants', meta=['gameId', 'seasonId', 'gameDuration'])
bans_matchdat_df=pd.json_normalize(data=r.json(), record_path=['teams', 'bans'])

# Rename bans_matchdat_df columns
bans_matchdat_df.columns = ['Banned Champion Id', 'pickTurn']

# Define a function for the pictTurn column value for the part_matchdat_df
def f(x):
    if x['participantId'] == 1: return 1
    elif x['participantId'] == 2: return 4
    elif x['participantId'] == 3: return 5
    elif x['participantId'] == 4: return 8
    elif x['participantId'] == 5: return 9
    elif x['participantId'] == 6: return 2
    elif x['participantId'] == 7: return 3
    elif x['participantId'] == 8: return 6
    elif x['participantId'] == 9: return 7
    elif x['participantId'] == 10: return 10
    
# Create pickTurn column for part_matchdat_df
part_matchdat_df['pickTurn'] = part_matchdat_df.apply(f, axis=1)

 
# Merge the two dataframes together on pickTurn
match_data_df = pd.merge(part_matchdat_df, bans_matchdat_df, on= 'pickTurn')

# Drop all rows not containing Nasus
indexNames = match_data_df[match_data_df['championId'] != 75].index
match_data_df.drop(indexNames, axis=0, inplace=True)



In [8]:
# Fill NaNs if exist for following columns
match_data_df[['stats.firstBloodAssist']].fillna('False', inplace=True)
match_data_df[['stats.firstTowerKill']].fillna('False', inplace=True)
match_data_df[['stats.firstTowerAssist']].fillna('False', inplace=True)
match_data_df[['stats.firstInhibitorKill']].fillna('False', inplace=True)
match_data_df[['stats.firstInhibitorAssist']].fillna('False', inplace=True)


c:\users\user\anaconda3\envs\streamlit\lib\site-packages\pandas\core\frame.py:4316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [11]:
match_data_df.isna().sum()

participantId                                 0
teamId                                        0
championId                                    0
spell1Id                                      0
spell2Id                                      0
stats.participantId                           0
stats.win                                     0
stats.item0                                   0
stats.item1                                   0
stats.item2                                   0
stats.item3                                   0
stats.item4                                   0
stats.item5                                   0
stats.item6                                   0
stats.kills                                   0
stats.deaths                                  0
stats.assists                                 0
stats.largestKillingSpree                     0
stats.largestMultiKill                        0
stats.killingSprees                           0
stats.longestTimeSpentLiving            

In [12]:
#### Spell Id columns
# Create two function to map the spell codes
def Summ_Spell_Names_ID1(Dataframe):
    if Dataframe['spell1Id']==21: return "Barrier"
    elif Dataframe['spell1Id']==1: return "Cleanse"
    elif Dataframe['spell1Id']==14: return "Ignite"
    elif Dataframe['spell1Id']==3: return "Exhaust"
    elif Dataframe['spell1Id'] == 4: return "Flash"
    elif Dataframe['spell1Id'] == 6: return "Ghost"
    elif Dataframe['spell1Id'] == 7: return "Heal"
    elif Dataframe['spell1Id'] == 11: return "Smite"
    elif Dataframe['spell1Id'] == 12: return "Teleport"

def Summ_Spell_Names_ID2(Dataframe):
    if Dataframe['spell2Id']==21: return "Barrier"
    elif Dataframe['spell2Id']==1: return "Cleanse"
    elif Dataframe['spell2Id']==14: return "Ignite"
    elif Dataframe['spell2Id']==3: return "Exhaust"
    elif Dataframe['spell2Id'] == 4: return "Flash"
    elif Dataframe['spell2Id'] == 6: return "Ghost"
    elif Dataframe['spell2Id'] == 7: return "Heal"
    elif Dataframe['spell2Id'] == 11: return "Smite"
    elif Dataframe['spell2Id'] == 12: return "Teleport"

    
# Create a column for Spell1Name and apply the function
match_data_df['Spell1Name'] = match_data_df.apply(Summ_Spell_Names_ID1, axis=1)


# Create a column for Spell2Name and apply the function
match_data_df['Spell2Name'] = match_data_df.apply(Summ_Spell_Names_ID2, axis=1)


# Create a summoner spells column which is a list of both spell1name and spell2name
match_data_df['Summoner Spells'] = match_data_df[['Spell1Name','Spell2Name']].values.tolist()   


In [13]:
# Import MLB
from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate
mlb = MultiLabelBinarizer(sparse_output=True)

# Binarizes the 'Sorted Summoner Spells' column and joins it to the original dataframe, while also dropping the original column
match_data_df = match_data_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(match_data_df.pop('Summoner Spells')),
                index=match_data_df.index,
                columns=mlb.classes_))

# Convert 'stats.win' dtype to binary integer
match_data_df['stats.win'] = match_data_df['stats.win'].astype(int)

In [14]:
# Import item.json file to create a dictionary for the item files
with open('item.json') as f:
    items_data = json.load(f)
key_dictionary = {}
for key in items_data['data']:
    key_dictionary.update({key:items_data['data'][key]['name']})

# add a value for none
key_dictionary.update({'0':'None'})

# Convert the item column to str
match_data_df['stats.item0'] = match_data_df['stats.item0'].astype(str)
match_data_df['stats.item1'] = match_data_df['stats.item1'].astype(str)
match_data_df['stats.item2'] = match_data_df['stats.item2'].astype(str)
match_data_df['stats.item3'] = match_data_df['stats.item3'].astype(str)
match_data_df['stats.item4'] = match_data_df['stats.item4'].astype(str)
match_data_df['stats.item5'] = match_data_df['stats.item5'].astype(str)
match_data_df['stats.item6'] = match_data_df['stats.item6'].astype(str)

# map the names using the dictionary
match_data_df = match_data_df.replace({"stats.item0": key_dictionary, "stats.item1": key_dictionary, "stats.item2": key_dictionary, "stats.item3": key_dictionary, "stats.item4": key_dictionary, "stats.item5": key_dictionary, "stats.item6": key_dictionary})

# Create an items list
match_data_df['Item List'] = match_data_df[['stats.item0', 'stats.item1', 'stats.item2', 'stats.item3', 'stats.item4', 'stats.item5', 'stats.item6']].values.tolist()


In [15]:
mlb = MultiLabelBinarizer(sparse_output=True)

# Binarizes the 'Item List' column and joins it to the original dataframe, while also dropping the original column
match_data_df = match_data_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(match_data_df.pop('Item List')),
                index=match_data_df.index,
                columns=mlb.classes_))


In [16]:
# Create fraction of magic damage done to champions
match_data_df['Fraction Magic Damage Delt to Champions'] = match_data_df['stats.magicDamageDealtToChampions']/match_data_df['stats.magicDamageDealt']

# Create fraction of Physical damage done to champions
match_data_df['Fraction Physical Damage Delt to Champions'] = match_data_df['stats.physicalDamageDealtToChampions']/match_data_df['stats.physicalDamageDealt']

# Create fraction of true damage done to champions
match_data_df['Fraction True Damage Delt to Champions'] = match_data_df['stats.trueDamageDealtToChampions']/match_data_df['stats.trueDamageDealt']

# Create fractional column for damage dealt to turrets
match_data_df['Fraction of damage dealt to turrets'] = match_data_df['stats.damageDealtToTurrets']/match_data_df['stats.damageDealtToObjectives']


In [17]:
# Import runesReforged.json file in order to create a dictionary to map the runes
with open('runesReforged.json') as f:
    rune_data = json.load(f)

# Create a key dictionary for runes
rune_key_dictionary = {}
for key1 in range(0,len(rune_data)):
    for key2 in range(0,len(rune_data[key1]['slots'])):
        for key3 in range(0,len(rune_data[key1]['slots'][key2]['runes'])):
            rune_key_dictionary.update({rune_data[key1]['slots'][key2]['runes'][key3]['id']:rune_data[key1]['slots'][key2]['runes'][key3]['key']})
            
# map the names using the dictionary
match_data_df = match_data_df.replace({"stats.perk0": rune_key_dictionary, "stats.perk1": rune_key_dictionary, 
                                           "stats.perk2": rune_key_dictionary, "stats.perk3": rune_key_dictionary, 
                                           "stats.perk4": rune_key_dictionary, "stats.perk5": rune_key_dictionary})            

# Convert perks to string just incase
match_data_df[['stats.perk0', 'stats.perk1', 'stats.perk2', 'stats.perk3', 'stats.perk4', 'stats.perk5']] = match_data_df[['stats.perk0', 'stats.perk1', 'stats.perk2', 'stats.perk3', 'stats.perk4', 'stats.perk5']].astype(str)


# Combine runes into a list column
match_data_df['Rune List'] = match_data_df[['stats.perk0', 'stats.perk1', 'stats.perk2', 'stats.perk3', 'stats.perk4', 'stats.perk5']].values.tolist()


In [18]:
# Instantiate
mlb = MultiLabelBinarizer(sparse_output=True)

# Binarizes the 'Item List' column and joins it to the original dataframe, while also dropping the original column
match_data_df = match_data_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(match_data_df.pop('Rune List')),
                index=match_data_df.index,
                columns=mlb.classes_))


In [19]:
# Convert boolean columns to integers
match_data_df[['stats.firstBloodKill', 'stats.firstBloodAssist', 'stats.firstTowerKill', 
                 'stats.firstTowerAssist', 'stats.firstInhibitorKill', 'stats.firstInhibitorAssist']] = match_data_df[['stats.firstBloodKill', 'stats.firstBloodAssist', 'stats.firstTowerKill', 
                 'stats.firstTowerAssist', 'stats.firstInhibitorKill', 'stats.firstInhibitorAssist']].astype(int)

In [20]:
# Set up the right feature columns
model_col = ['stats.kills','stats.deaths','stats.assists','stats.largestKillingSpree','stats.largestMultiKill','stats.killingSprees',
'stats.longestTimeSpentLiving','stats.doubleKills','stats.tripleKills','stats.quadraKills','stats.pentaKills','stats.magicDamageDealt',
'stats.physicalDamageDealt','stats.trueDamageDealt','stats.largestCriticalStrike','stats.totalHeal','stats.totalUnitsHealed',
'stats.damageSelfMitigated','stats.damageDealtToObjectives','stats.visionScore','stats.timeCCingOthers','stats.totalDamageTaken',
'stats.goldEarned','stats.turretKills','stats.inhibitorKills','stats.totalMinionsKilled','stats.neutralMinionsKilled',
'stats.neutralMinionsKilledTeamJungle','stats.neutralMinionsKilledEnemyJungle','stats.totalTimeCrowdControlDealt','stats.visionWardsBoughtInGame',
'stats.wardsPlaced','stats.wardsKilled','stats.firstBloodKill','stats.firstBloodAssist','stats.firstTowerKill','stats.firstTowerAssist',
'stats.firstInhibitorKill','stats.firstInhibitorAssist','gameDuration', 'Cleanse','Exhaust','Flash','Ghost','Heal','Ignite','Smite','Teleport',"'Your Cut'",
'Abyssal Mask','Adaptive Helm','Aegis of the Legion','Aether Wisp','Amplifying Tome',"Archangel's Staff",'Ardent Censer','B. F. Sword',"Bami's Cinder",
"Banshee's Veil","Berserker's Greaves",'Bilgewater Cutlass','Black Cleaver','Blade of the Ruined King','Blasting Wand','Bloodthirster','Boots of Mobility',
'Boots of Speed','Boots of Swiftness','Bramble Vest','Broken Stopwatch','Bulwark of the Mountain','Catalyst of Aeons',"Caulfield's Warhammer",'Chain Vest',
'Cloak of Agility','Cloth Armor','Commencing Stopwatch','Control Ward','Corrupting Potion','Crystalline Bracer','Cull','Dagger','Dark Seal',
"Dead Man's Plate","Death's Dance","Doran's Blade","Doran's Ring","Doran's Shield",'Duskblade of Draktharr','Edge of Night','Elixir of Iron',
'Elixir of Wrath','Enchantment: Bloodrazor','Enchantment: Cinderhulk','Enchantment: Runic Echoes','Enchantment: Warrior','Essence Reaver',
"Executioner's Calling",'Eye of the Herald','Faerie Charm','Farsight Alteration','Fiendish Codex','Forbidden Idol','Frostfang','Frozen Fist','Frozen Heart',
'Frozen Mallet','Gargoyle Stoneplate',"Giant's Belt",'Glacial Shroud','Guardian Angel',"Guinsoo's Rageblade",'Haunting Guise','Health Potion','Hexdrinker',
'Hextech GLP-800','Hextech Gunblade','Hextech Protobelt-01','Hextech Revolver',"Hunter's Machete","Hunter's Talisman",'Iceborn Gauntlet','Infinity Edge',
'Ionian Boots of Lucidity',"Jaurim's Fist",'Kindlegem','Kircheis Shard',"Knight's Vow",'Last Whisper',"Liandry's Torment",'Lich Bane',
'Locket of the Iron Solari','Long Sword',"Lord Dominik's Regards",'Lost Chapter',"Luden's Echo","Luden's Pulse",'Manamune','Maw of Malmortius',
"Mejai's Soulstealer",'Mercurial Scimitar',"Mercury's Treads",'Minion Dematerializer','Morellonomicon','Mortal Reminder','Muramana',"Nashor's Tooth",
'Needlessly Large Rod','Negatron Cloak','Ninja Tabi','Null-Magic Mantle','Oblivion Orb','Oracle Lens','Pauldrons of Whiterock','Perfectly Timed Stopwatch',
'Phage','Phantom Dancer','Pickaxe','Quicksilver Sash',"Rabadon's Deathcap","Randuin's Omen",'Rapid Firecannon','Ravenous Hydra','Recurve Bow','Redemption',
'Refillable Potion','Rejuvenation Bead','Righteous Glory','Rod of Ages','Ruby Crystal','Runesteel Spaulders',"Rylai's Crystal Scepter",'Sanguine Blade',
'Sapphire Crystal',"Seeker's Armguard","Seraph's Embrace",'Serrated Dirk','Shard of True Ice','Sheen',"Shurelya's Reverie","Skirmisher's Sabre",
'Slightly Magical Boots',"Sorcerer's Shoes",'Spear of Shojin',"Spectre's Cowl",'Spellbinder','Spirit Visage',"Stalker's Blade",'Statikk Shiv',
"Sterak's Gage",'Stinger','Stopwatch','Stormrazor','Sunfire Cape','Tear of the Goddess','Thornmail','Tiamat','Titanic Hydra',
'Total Biscuit of Everlasting Will','Trinity Force','Trinity Fusion','Twin Shadows','Vampiric Scepter','Void Staff',"Warden's Mail",
'Warding Totem (Trinket)',"Warmog's Armor","Wit's End","Youmuu's Ghostblade",'Zeal',"Zeke's Convergence","Zhonya's Hourglass",
'Fraction Magic Damage Delt to Champions','Fraction Physical Damage Delt to Champions','Fraction True Damage Delt to Champions',
'Fraction of damage dealt to turrets','AbsoluteFocus','ApproachVelocity','ArcaneComet','BiscuitDelivery','BonePlating','Celerity','CheapShot',
'Conditioning','Conqueror','CosmicInsight','CoupDeGrace','CutDown','DarkHarvest','Demolish','Electrocute','EyeballCollection','FleetFootwork','FontOfLife',
'FuturesMarket','GatheringStorm','GhostPoro','GlacialAugment','GraspOfTheUndying','Guardian','HailOfBlades','HextechFlashtraption','IngeniousHunter',
'LastStand','LegendAlacrity','LegendBloodline','LegendTenacity','LethalTempo','MagicalFootwear','ManaflowBand','MasterKey','MinionDematerializer',
'NimbusCloak','NullifyingOrb','Overgrowth','Overheal','PerfectTiming','PhaseRush','Predator','PresenceOfMind','PressTheAttack','RavenousHunter',
'RelentlessHunter','Revitalize','Scorch','SecondWind','ShieldBash','SuddenImpact','SummonAery','TasteOfBlood','TimeWarpTonic','Transcendence','Triumph',
'UltimateHunter','Unflinching','UnsealedSpellbook','Waterwalking','ZombieWard']


In [21]:
missing_features = {}
for feature in model_col:
    if feature in match_data_df.columns:
        pass # Don't need to do anything if the feature exists
    else:
        # Update the dictionary
        missing_features.update({feature:[0]})
        # create a column out of that feature and input 0

#Turn the dictionary into a dataframe
missing_df = pd.DataFrame.from_dict(missing_features)

In [27]:
# Stick dataframes together
result_df = pd.concat([match_data_df, missing_df.reindex(match_data_df.index)], axis=1).fillna(0)

In [29]:
# Set up variable to feed into model
X = result_df[model_col].reset_index(drop=True)
print(X.shape)
display(X.head())

(1, 278)


,stats.kills,stats.deaths,stats.assists,stats.largestKillingSpree,stats.largestMultiKill,stats.killingSprees,stats.longestTimeSpentLiving,stats.doubleKills,stats.tripleKills,stats.quadraKills,stats.pentaKills,stats.magicDamageDealt,stats.physicalDamageDealt,stats.trueDamageDealt,stats.largestCriticalStrike,stats.totalHeal,stats.totalUnitsHealed,stats.damageSelfMitigated,stats.damageDealtToObjectives,stats.visionScore,stats.timeCCingOthers,stats.totalDamageTaken,stats.goldEarned,stats.turretKills,stats.inhibitorKills,stats.totalMinionsKilled,stats.neutralMinionsKilled,stats.neutralMinionsKilledTeamJungle,stats.neutralMinionsKilledEnemyJungle,stats.totalTimeCrowdControlDealt,stats.visionWardsBoughtInGame,stats.wardsPlaced,stats.wardsKilled,stats.firstBloodKill,stats.firstBloodAssist,stats.firstTowerKill,stats.firstTowerAssist,stats.firstInhibitorKill,stats.firstInhibitorAssist,gameDuration,Cleanse,Exhaust,Flash,Ghost,Heal,Ignite,Smite,Teleport,'Your Cut',Abyssal Mask,Adaptive Helm,Aegis of the Legion,Aether Wisp,Amplifying Tome,Archangel's Staff,Ardent Censer,B. F. Sword,Bami's Cinder,Banshee's Veil,Berserker's Greaves,Bilgewater Cutlass,Black Cleaver,Blade of the Ruined King,Blasting Wand,Bloodthirster,Boots of Mobility,Boots of Speed,Boots of Swiftness,Bramble Vest,Broken Stopwatch,Bulwark of the Mountain,Catalyst of Aeons,Caulfield's Warhammer,Chain Vest,Cloak of Agility,Cloth Armor,Commencing Stopwatch,Control Ward,Corrupting Potion,Crystalline Bracer,Cull,Dagger,Dark Seal,Dead Man's Plate,Death's Dance,Doran's Blade,Doran's Ring,Doran's Shield,Duskblade of Draktharr,Edge of Night,Elixir of Iron,Elixir of Wrath,Enchantment: Bloodrazor,Enchantment: Cinderhulk,Enchantment: Runic Echoes,Enchantment: Warrior,Essence Reaver,Executioner's Calling,Eye of the Herald,Faerie Charm,Farsight Alteration,Fiendish Codex,Forbidden Idol,Frostfang,Frozen Fist,Frozen Heart,Frozen Mallet,Gargoyle Stoneplate,Giant's Belt,Glacial Shroud,Guardian Angel,Guinsoo's Rageblade,Haunting Guise,Health Potion,Hexdrinker,Hextech GLP-800,Hextech Gunblade,Hextech Protobelt-01,Hextech Revolver,Hunter's Machete,Hunter's Talisman,Iceborn Gauntlet,Infinity Edge,Ionian Boots of Lucidity,Jaurim's Fist,Kindlegem,Kircheis Shard,Knight's Vow,Last Whisper,Liandry's Torment,Lich Bane,Locket of the Iron Solari,Long Sword,Lord Dominik's Regards,Lost Chapter,Luden's Echo,Luden's Pulse,Manamune,Maw of Malmortius,Mejai's Soulstealer,Mercurial Scimitar,Mercury's Treads,Minion Dematerializer,Morellonomicon,Mortal Reminder,Muramana,Nashor's Tooth,Needlessly Large Rod,Negatron Cloak,Ninja Tabi,Null-Magic Mantle,Oblivion Orb,Oracle Lens,Pauldrons of Whiterock,Perfectly Timed Stopwatch,Phage,Phantom Dancer,Pickaxe,Quicksilver Sash,Rabadon's Deathcap,Randuin's Omen,Rapid Firecannon,Ravenous Hydra,Recurve Bow,Redemption,Refillable Potion,Rejuvenation Bead,Righteous Glory,Rod of Ages,Ruby Crystal,Runesteel Spaulders,Rylai's Crystal Scepter,Sanguine Blade,Sapphire Crystal,Seeker's Armguard,Seraph's Embrace,Serrated Dirk,Shard of True Ice,Sheen,Shurelya's Reverie,Skirmisher's Sabre,Slightly Magical Boots,Sorcerer's Shoes,Spear of Shojin,Spectre's Cowl,Spellbinder,Spirit Visage,Stalker's Blade,Statikk Shiv,Sterak's Gage,Stinger,Stopwatch,Stormrazor,Sunfire Cape,Tear of the Goddess,Thornmail,Tiamat,Titanic Hydra,Total Biscuit of Everlasting Will,Trinity Force,Trinity Fusion,Twin Shadows,Vampiric Scepter,Void Staff,Warden's Mail,Warding Totem (Trinket),Warmog's Armor,Wit's End,Youmuu's Ghostblade,Zeal,Zeke's Convergence,Zhonya's Hourglass,Fraction Magic Damage Delt to Champions,Fraction Physical Damage Delt to Champions,Fraction True Damage Delt to Champions,Fraction of damage dealt to turrets,AbsoluteFocus,ApproachVelocity,ArcaneComet,BiscuitDelivery,BonePlating,Celerity,CheapShot,Conditioning,Conqueror,CosmicInsight,CoupDeGrace,CutDown,DarkHarvest,Demolish,Electrocute,EyeballCollection,FleetFootwork,FontOfLife,FuturesMarket,GatheringStorm,GhostPoro,GlacialAugment,GraspOfTheUndying,Guard

In [35]:
model.predict_proba(X)[0]

c:\users\user\anaconda3\envs\streamlit\lib\site-packages\sklearn\utils\validation.py:508: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


IndexError: index 1 is out of bounds for axis 0 with size 1